#**Integrantes**

  Lucca Costa Lima D'Avila - 171903

  Henrique Casarini Firmino - 180185

  Carlos Eduardo Ferreira da Silva - 223748


# Neste trabalho, utilizaremos um algoritmo que verifica a quantidade de numeros primos em um intervalo de 2 a k.
# Nele, aplicaremos dois dos conjuntos propostos (bcast/reduce & send/recv), bem como o timeit para verificar o tempo de execução de cada loop e com isso analisar os resultados obtidos em cada um dos conjuntos.
# O código está organizado em apenas workers.

In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 24.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp37-cp37m-linux_x86_64.whl size=2185774 sha256=27c03568d06a7b4cb518f262a4be76f389a96ab131b0f04a599e3f107c264d66
  Stored in directory: /root/.cache/pip/wheels/99/54/29/187b5768bbb7beeab6753bc30acf56f35bc8ca9c214a31e173
Successfully built mpi4py


# A seguir, temos a aplicação do primeiro conjunto (bcast/reduce)

In [ ]:
%%writefile bcastreduce.py

import numpy as np
import platform
import sys
from mpi4py import MPI

comm = MPI.COMM_WORLD
id = comm.Get_rank()
size = comm.Get_size()

# Encontra os numeros primos entre 2 e k 
# Inicializando k
k = 20
# Define a lista de primos entre 2 e k
S_k = []
# Define uma lista para guardar os numeros que nao sao primos entre 2 e k
not_prime = []
# Define uma lista de primos entre k e k**2
S_k2 = []

# Conta o numero de primos de 2 até k
for i in range(2, k+1):
    if i not in not_prime:
        S_k.append(i)
        for j in range(i*i, k+1, i):
            not_prime.append(j)

# Encontra o numero de primos entre k e k**2 por paralelização do loop
b=(k**2-k)/size
for n in range(int(k+(id)*b),int(k+(id+1)*b)):
    counter = 0
    for i in range(len(S_k)):
        if (n % S_k[i]) == 0:
            break
        else:
            counter = counter + 1
    if (counter==len(S_k)):
        S_k2.append(n)

# Avalia a quantidade de primos nas duas listas
processor_num_primes = len(S_k2)
original_num_primes = len(S_k)

# Faz o broadcast da quantidade de primos e calcula as uniões de conjuntos de inteiros.
countb = 0
totalb = 0
for i in range(0, size):
    countb = comm.bcast(S_k2, i)
    totalb = totalb + len(countb)
total = comm.reduce(totalb, MPI.BOR,0)

if (id == 0):
    print ("Numero total de processos: ", size)
print ("Processo ", id + 1, "esta calculando:", S_k2)
print ("Processo ", id + 1, "esta calculando esta quantidade de primos:", processor_num_primes)
print("Processo ", id + 1, "tem o seguinte intervalo:", (int(k + (id) * b), int(k + (id + 1) * b)))
if (id == 0):
    print("Total de numeros primos encontrados no intervalo entre", 2, "e", k * k, "foi de:", total + original_num_primes)
print('')

Writing bcastreduce.py


# A partir daqui, temos a execução do código com 1, 2, 4 e 8 processos, bem como a aplicação do timeit para verificar o tempo de execução de cada um.

In [ ]:
import timeit

In [ ]:
%%timeit -n 1 -r 1
!mpirun --allow-run-as-root -np 1 python ./bcastreduce.py

Numero total de processos:  1
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  1 esta calculando esta quantidade de primos: 70
Processo  1 tem o seguinte intervalo: (20, 400)
Total de numeros primos encontrados no intervalo entre 2 e 400 foi de: 78

417 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 2 -r 1
!mpirun --allow-run-as-root -np 2 python ./bcastreduce.py

Numero total de processos:  2
Processo  2 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  2 esta calculando esta quantidade de primos: 32
Processo  2 tem o seguinte intervalo: (210, 400)

Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  1 esta calculando esta quantidade de primos: 38
Processo  1 tem o seguinte intervalo: (20, 210)
Total de numeros primos encontrados no intervalo entre 2 e 400 foi de: 78

Numero total de processos:  2
Processo  2 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  2 esta calculando esta quantidade de primos: 32
Processo  

In [ ]:
%%timeit -n 4 -r 1
!mpirun --allow-run-as-root -np 4 python ./bcastreduce.py

Processo  4 esta calculando: [307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  4 esta calculando esta quantidade de primos: 16
Processo  4 tem o seguinte intervalo: (305, 400)

Processo  2 esta calculando: [127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  2 esta calculando esta quantidade de primos: 16
Processo  2 tem o seguinte intervalo: (115, 210)

Numero total de processos:  4
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]
Processo  1 esta calculando esta quantidade de primos: 22
Processo  3 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293]
Processo  1 tem o seguinte intervalo: (20, 115)
Total de numeros primos encontrados no intervalo entre 2 e 400 foi de: 78

Processo  3 esta calculando esta quantidade de primos: 16
Processo  3 tem o seguinte intervalo: (210, 305)

Processo  3 

In [ ]:
%%timeit -n 8 -r 1
!mpirun --allow-run-as-root -np 8 python ./bcastreduce.py

Processo  4 esta calculando: [163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  8 esta calculando: [353, 359, 367, 373, 379, 383, 389, 397]
Processo  6 esta calculando: [257, 263, 269, 271, 277, 281, 283, 293]
Processo  7 esta calculando: [307, 311, 313, 317, 331, 337, 347, 349]
Processo  7 esta calculando esta quantidade de primos: 8
Processo  7 tem o seguinte intervalo: (305, 352)

Processo  6 esta calculando esta quantidade de primos: 8
Processo  6 tem o seguinte intervalo: (257, 305)

Processo  3 esta calculando: [127, 131, 137, 139, 149, 151, 157]
Processo  2 esta calculando: [67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]
Processo  2 esta calculando esta quantidade de primos: 12
Processo  3 esta calculando esta quantidade de primos: 7
Processo  4 esta calculando esta quantidade de primos: 9
Processo  2 tem o seguinte intervalo: (67, 115)
Processo  3 tem o seguinte intervalo: (115, 162)
Processo  4 tem o seguinte intervalo: (162, 210)


Processo  8 esta calculando esta 

# A seguir, temos a aplicação do segundo conjunto (send/recv)

In [254]:
%%writefile sendrecv.py

import numpy as np
import platform
import sys
from mpi4py import MPI

comm = MPI.COMM_WORLD
id = comm.Get_rank()
size = comm.Get_size()

# Encontra os numeros primos entre 2 e k 
# Inicializando k
k = 20
# Define a lista de primos entre 2 e k
S_k = []
# Define uma lista para guardar os numeros que nao sao primos entre 2 e k
not_prime = []
# Define uma lista de primos entre k e k**2
S_k2 = []

# Conta o numero de primos de 2 até k
for i in range(2, k+1):
    if i not in not_prime:
        S_k.append(i)
        for j in range(i*i, k+1, i):
            not_prime.append(j)

# Encontra o numero de primos entre k e k**2 por paralelização do loop
b=(k**2-k)/size
for n in range(int(k+(id)*b),int(k+(id+1)*b)):
    counter = 0
    for i in range(len(S_k)):
        if (n % S_k[i]) == 0:
            break
        else:
            counter = counter + 1
    if (counter==len(S_k)):
        S_k2.append(n)

# Avalia a quantidade de primos nas duas listas
processor_num_primes = len(S_k2)
original_num_primes = len(S_k)

# Faz o broadcast da quantidade de primos e calcula as uniões de conjuntos de inteiros.
total = S_k2

if id == 1:
  total = S_k2
  comm.send(total, dest = 1, tag = 10)
elif id == 1:
  total = comm.recv(source = 0, tag = 10)

if (id == 0):
    print ("Numero total de processos: ", size)
print ("Processo ", id + 1, "esta calculando:", S_k2)
print ("Processo ", id + 1, "esta calculando esta quantidade de primos:", processor_num_primes)
print("Processo ", id + 1, "tem o seguinte intervalo:", (int(k + (id) * b), int(k + (id + 1) * b)))
#print("Total de numeros primos encontrados no intervalo entre", 2, "e", k * k, "foi de:", S_k + total)
#if (id == 1):
#    print("Total de numeros primos encontrados no intervalo entre", 2, "e", k * k, "foi de:", len(S_k) + len(total) )
#print('')

Overwriting sendrecv.py


In [257]:
%%timeit -n 1 -r 1
!mpirun --allow-run-as-root -np 1 python ./sendrecv.py

Numero total de processos:  1
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  1 esta calculando esta quantidade de primos: 70
Processo  1 tem o seguinte intervalo: (20, 400)
4.58 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [258]:
%%timeit -n 1 -r 1
!mpirun --allow-run-as-root -np 2 python ./sendrecv.py

Processo  2 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  2 esta calculando esta quantidade de primos: 32
Processo  2 tem o seguinte intervalo: (210, 400)
Numero total de processos:  2
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  1 esta calculando esta quantidade de primos: 38
Processo  1 tem o seguinte intervalo: (20, 210)
4.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [259]:
%%timeit -n 1 -r 1
!mpirun --allow-run-as-root -np 4 python ./sendrecv.py

Processo  3 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293]
Processo  3 esta calculando esta quantidade de primos: 16
Processo  3 tem o seguinte intervalo: (210, 305)
Processo  2 esta calculando: [127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  4 esta calculando: [307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
Processo  2 esta calculando esta quantidade de primos: 16
Processo  2 tem o seguinte intervalo: (115, 210)
Processo  4 esta calculando esta quantidade de primos: 16
Processo  4 tem o seguinte intervalo: (305, 400)
Numero total de processos:  4
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]
Processo  1 esta calculando esta quantidade de primos: 22
Processo  1 tem o seguinte intervalo: (20, 115)
7.38 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [261]:
%%timeit -n 1 -r 1
!mpirun --allow-run-as-root -np 8 python ./sendrecv.py

Processo  4 esta calculando: [163, 167, 173, 179, 181, 191, 193, 197, 199]
Processo  4 esta calculando esta quantidade de primos: 9
Processo  4 tem o seguinte intervalo: (162, 210)
Processo  6 esta calculando: [257, 263, 269, 271, 277, 281, 283, 293]
Processo  6 esta calculando esta quantidade de primos: 8
Processo  2 esta calculando: [67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]
Processo  2 esta calculando esta quantidade de primos: 12
Processo  2 tem o seguinte intervalo: (67, 115)
Processo  3 esta calculando: [127, 131, 137, 139, 149, 151, 157]
Processo  3 esta calculando esta quantidade de primos: 7
Processo  3 tem o seguinte intervalo: (115, 162)
Numero total de processos:  8
Processo  1 esta calculando: [23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
Processo  1 esta calculando esta quantidade de primos: 10
Processo  1 tem o seguinte intervalo: (20, 67)
Processo  5 esta calculando: [211, 223, 227, 229, 233, 239, 241, 251]
Processo  5 esta calculando esta quantidade de primos: 8


#***CONCLUSÃO***

# Analisando a execução de ambos os conjuntos (bcast/reduce & send/recev) neste algoritmo, que encontra a quantidade de números primos em um intervalo de 2 a k, e o tempo de execução de cada loop, podemos observar que o tempo de execução aumenta conforme aumentamos a quantidade de processos. Porém, conforme dividimos os números a cada processo, eles deveriam otimizar o tempo.
# Isso deve ocorrer pelo fato do broadcast transmitir a informação para mais processos a cada vez. Parte do aumento de tempo de execução também pode vir do próprio colab, algo parecido com o que ocorreu durante a AC1.